In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import yfinance as yf

In [ ]:
# Define the date range for data collection
start = '2019-01-01'
end = '2023-12-31'
test_start = '2024-01-01'
test_end = '2024-03-01'

In [ ]:
# Number of days to forecast
forecast_days = 20

# Define the list of tickers
ticker = ['TECHM.NS', 'M&M.NS', 'WIPRO.NS', 'SBIN.NS', 'BAJFINANCE.NS', 'INFY.NS', 'KOTAKBANK.NS']

In [ ]:
# Function to calculate technical indicators
def add_technical_indicators(data):
    # Simple Moving Average (SMA)
    data['SMA'] = data['Close'].rolling(window=14).mean()

    # Exponential Moving Average (EMA)
    data['EMA'] = data['Close'].ewm(span=14, adjust=False).mean()

    # Relative Strength Index (RSI)
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14, min_periods=1).mean()
    avg_loss = loss.rolling(window=14, min_periods=1).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # Fill NaN values
    data.fillna(method='bfill', inplace=True)
    return data

In [ ]:
for company in ticker:
    # Download stock data
    data = yf.download(company, start=start, end=end)

    # Add technical indicators
    data = add_technical_indicators(data)

    # Select features: Closing price, SMA, EMA, RSI
    features = data[['Close', 'SMA', 'EMA', 'RSI']].values

    # Normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(features)

    # Prepare training data
    X_train, y_train = [], []
    for i in range(60, len(scaled_data) - forecast_days):
        X_train.append(scaled_data[i-60:i])
        y_train.append(scaled_data[i + forecast_days - 1, 0])  # Predict the future closing price

    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshape for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Predicting the next price

    # Compile and train the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32)

    # Prepare testing data and make predictions
    inputs = scaled_data[len(scaled_data) - len(data) - 60:].reshape(-1, scaled_data.shape[1])
    X_test = []
    for i in range(60, len(inputs)):
        X_test.append(inputs[i-60:i])

    X_test = np.array(X_test)

    # Check the dimensions of X_test before reshaping
    print(f"Shape of X_test before reshaping: {X_test.shape}")

    # If X_test is 2D, reshape to 3D
    if len(X_test.shape) == 2:
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Now X_test should be 3D, compatible with LSTM input
    print(f"Shape of X_test after reshaping: {X_test.shape}")
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    predicted_prices = model.predict(X_test)
    predicted_prices = scaler.inverse_transform(np.hstack((predicted_prices, np.zeros((predicted_prices.shape[0], scaled_data.shape[1] - 1)))))[:, 0]  # Reverse scaling only for the first column (Close price)

    # Download actual test data for comparison
    compare_data = yf.download(company, start=test_start, end=test_end)
    comp_data = compare_data['Close'].iloc[:forecast_days]
    predicted_dates = comp_data.index[:forecast_days]

    # Plotting results
    plt.figure(figsize=(14, 5))
    plt.plot(data.index[-60:], data['Close'].values[-60:], color='blue', label='Historical Prices')
    plt.plot(predicted_dates, predicted_prices, color='red', label='Predicted Prices')
    plt.title(f'Stock Price Prediction: {company}')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()


[*********************100%***********************]  1 of 1 completed

Epoch 1/50



<ipython-input-4-b945e064feaa>:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/37 ━━━━━━━━━━━━━━━━━━━━ 1:09 2s/step - loss: 0.2971

In [ ]:
from google.colab import drive
drive.mount('/content/drive')